<a href="https://colab.research.google.com/github/obliquesignal/algo-trading-bootcamp/blob/master/5a_Multilayer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
from keras import models
from keras import layers

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
from datetime import datetime

import matplotlib.pyplot as plt
plt.style.use('seaborn')

# Import and Process Data

In [ ]:
start = datetime(1982, 1, 1)
end = datetime(2020, 2, 29)

recession = pdr.DataReader('USREC', 'fred', start, end) #NBER business cycle classification
yield_curve = pdr.DataReader('T10Y3MM', 'fred', start, end) #Difference between the 3 month and 10 year treasury yields
unemployment = pdr.DataReader('UNRATE', 'fred', start, end) #Unemployment rate
industrial_capacity = pdr.DataReader('TCU', 'fred', start, end) #Total industrial capacity utilization

In [ ]:
#Create target dataframe
target = recession[1:] #Align target and feature rows
target.head()

,USREC
DATE,
1982-02-01,1
1982-03-01,1
1982-04-01,1
1982-05-01,1
1982-06-01,1


In [ ]:
#Percentage of time the US economy was in recession since 1982. This is an imbalanced dataset
round(target['USREC'].sum()/target['USREC'].count()*100, 2)

9.63

In [ ]:
#Create features dataframe
features = pd.DataFrame()
features['curve'] = yield_curve['T10Y3MM'].diff() #Difference between the yields of the 3 month bill and the 10 year note
features['unemployment'] = unemployment['UNRATE'].diff()
features['industrial'] = industrial_capacity['TCU'].diff()
features = features.dropna()
features.head()

,curve,unemployment,industrial
DATE,,,
1982-02-01,-1.52,0.3,1.3705
1982-03-01,0.40,0.1,-0.6713
1982-04-01,-0.02,0.3,-0.8309
1982-05-01,0.38,0.1,-0.6352
1982-06-01,0.31,0.2,-0.2984


In [ ]:
#Standardize data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

target = np.ravel(target) #Need to covert column vector into a 1-d Numpy array

scaler = StandardScaler() #Scales input data so that columns/features have a mean of 0 and a standard deviation of 1
features_scaled = scaler.fit_transform(features) 

#Split dataset into train and test subsets. Test size is 25% of the total dataset and the data are not shuffled to preserve temporal structure of timeseries

features_train, features_test, target_train, target_test = train_test_split(features_scaled, target, test_size = 0.25, shuffle=False, random_state=101)

# Build Network

In [ ]:
#Stacking layers to create a network. A dense layer is a fully connected layer
fcn = models.Sequential() 

#First hidden layer needs to specify the shape of the feature data it will receive from the input layer
fcn.add(layers.Dense(units=32, activation='relu', input_shape=(3,)))

#Hidden layer which has 32 neurons and uses the Relu activation function to learn nonlinearity. Does not need an input_shape parameter
fcn.add(layers.Dense(units=32, activation='relu'))

# Output layer which uses the Sigmoid function for binary classification
fcn.add(layers.Dense(units=1, activation='sigmoid'))

#Network will minimize the binary cross entropy loss function using the Adam optimizer and will evaluate success using accuracy metric
fcn.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

In [ ]:
fcn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                128       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,217
Trainable params: 1,217
Non-trainable params: 0
_________________________________________________________________


# Train Network

In [ ]:
fcn.fit(features_train, target_train, epochs=30)

Epoch 1/30
11/11 [==============================] - 2s 5ms/step - loss: 0.8008 - accuracy: 0.1491
Epoch 2/30
11/11 [==============================] - 0s 4ms/step - loss: 0.6997 - accuracy: 0.5175
Epoch 3/30
11/11 [==============================] - 0s 4ms/step - loss: 0.6269 - accuracy: 0.8421
Epoch 4/30
11/11 [==============================] - 0s 14ms/step - loss: 0.5690 - accuracy: 0.8889
Epoch 5/30
11/11 [==============================] - 0s 5ms/step - loss: 0.5219 - accuracy: 0.8743
Epoch 6/30
11/11 [==============================] - 0s 5ms/step - loss: 0.4789 - accuracy: 0.8713
Epoch 7/30
11/11 [==============================] - 0s 7ms/step - loss: 0.4416 - accuracy: 0.8713
Epoch 8/30
11/11 [==============================] - 0s 4ms/step - loss: 0.4078 - accuracy: 0.8743
Epoch 9/30
11/11 [==============================] - 0s 6ms/step - loss: 0.3759 - accuracy: 0.8772
Epoch 10/30
11/11 [==============================] - 0s 4ms/step - loss: 0.3493 - accuracy: 0.8860
Epoch 11/30
11/11 

# Evaluate Network

In [ ]:
fcn.evaluate(features_test, target_test)

4/4 [==============================] - 0s 6ms/step - loss: 0.0684 - accuracy: 0.9913


[0.0684034526348114, 0.991304337978363]

In [ ]:
#Use out-of-sample March data to predict recession
march = [[0.61, 0.9, -4.2351]]
fcn.predict(march).round(2)

1/1 [==============================] - 0s 340ms/step


array([[0.76]], dtype=float32)

# Build Network for Regression

In [ ]:
gdp = pdr.DataReader('A191RL1Q225SBEA', 'fred', start, end) #Annualized percentage change in real GDP from previous quarter

In [ ]:
#Create regression target dataframe
targetvalue = gdp
targetvalue = targetvalue[1:] #Get rid of the first row in order to align with the features matrix
targetvalue.head()      

,A191RL1Q225SBEA
DATE,
1982-04-01,1.8
1982-07-01,-1.5
1982-10-01,0.2
1983-01-01,5.4
1983-04-01,9.4


In [ ]:
#Downsample features dataframe and convert it into a quarterly frequency
qfeatures = features.resample('1q', label='right').last()
#Removes the last row
qmarch = qfeatures[-1:]
#Removes the last row to align with features dataframe
qfeatures = qfeatures[:-1]
qfeatures.tail()

,curve,unemployment,industrial
DATE,,,
2018-12-31,-0.33,0.1,-0.1275
2019-03-31,-0.12,0.0,-0.0052
2019-06-30,-0.15,-0.1,-0.0685
2019-09-30,0.13,-0.2,-0.3353
2019-12-31,0.05,0.0,-0.3139


In [ ]:
qfeatures_scaled = scaler.fit_transform(qfeatures) #Every feature now has a mean of 0 and a standard deviation of 1

#Split dataset into train and test subsets. Test size is 25% of the total dataset and the data are not shuffled to preserve temporal structure of timeseries

qfeatures_train, qfeatures_test, targetvalue_train, targetvalue_test = train_test_split(qfeatures_scaled, targetvalue, test_size = 0.25, shuffle=False, random_state=0)

In [ ]:
regression_fcn = models.Sequential() #Stacking layers to create a network. A dense layer is a fully connected layer

regression_fcn.add(layers.Dense(units=32, activation='relu', input_shape=(3,))) #First hidden layer needs to specify the shape of the feature data it will receive from the input layer

regression_fcn.add(layers.Dense(units=32, activation='relu')) #Hidden layer which has 32 neurons and uses the Relu activation function to learn nonlinearity. Does not need an input_shape parameter

regression_fcn.add(layers.Dense(units=1))# Output layer which does not use the Sigmoid function for binary classification

regression_fcn.compile(loss='mse', optimizer='adam', metrics='mae')#Network will minimize the mean square error loss function using the Adam optimizer and will evaluate success using mae metric

In [ ]:
regression_fcn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                128       
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,217
Trainable params: 1,217
Non-trainable params: 0
_________________________________________________________________


In [ ]:
regression_fcn.fit(qfeatures_train, targetvalue_train, epochs=30)

Epoch 1/30
4/4 [==============================] - 3s 7ms/step - loss: 16.2587 - mae: 3.4999
Epoch 2/30
4/4 [==============================] - 0s 7ms/step - loss: 15.4483 - mae: 3.3883
Epoch 3/30
4/4 [==============================] - 0s 7ms/step - loss: 14.6746 - mae: 3.2749
Epoch 4/30
4/4 [==============================] - 0s 6ms/step - loss: 13.9787 - mae: 3.1751
Epoch 5/30
4/4 [==============================] - 0s 6ms/step - loss: 13.4258 - mae: 3.0836
Epoch 6/30
4/4 [==============================] - 0s 7ms/step - loss: 12.8070 - mae: 2.9854
Epoch 7/30
4/4 [==============================] - 0s 6ms/step - loss: 12.3151 - mae: 2.9062
Epoch 8/30
4/4 [==============================] - 0s 6ms/step - loss: 11.8095 - mae: 2.8219
Epoch 9/30
4/4 [==============================] - 0s 6ms/step - loss: 11.3191 - mae: 2.7418
Epoch 10/30
4/4 [==============================] - 0s 5ms/step - loss: 10.8715 - mae: 2.6691
Epoch 11/30
4/4 [==============================] - 0s 6ms/step - loss: 10.4799 

In [ ]:
regression_fcn.evaluate(qfeatures_test, targetvalue_test)

2/2 [==============================] - 0s 13ms/step - loss: 6.6408 - mae: 1.6807


[6.64080810546875, 1.680739402770996]

In [ ]:
regression_fcn.predict(qmarch)

1/1 [==============================] - 0s 126ms/step


array([[1.9663868]], dtype=float32)